## Nobel API - check Wikidata and Nobel API
version. 0.2 changed api call and added **Vrandečić EWS**
* This [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)  

  * WIkidata property [Property:P8024](https://www.wikidata.org/wiki/Property:P8024)
  * [Developer xone nobelprize.org](https://www.nobelprize.org/about/developer-zone-2/)
      * maybe better API is [api.nobelprize.org/2.0/laureates](https://api.nobelprize.org/2.0/laureates)

In [1]:
from datetime import datetime
now = datetime.now()
print("Last run: ", datetime.now())

Last run:  2020-10-06 21:19:17.489509


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url ="https://api.nobelprize.org/2.0/laureates?offset=0&limit=1500"
dftot = pd.DataFrame()
r = http.request('GET', url)
data = json.loads(r.data)
dftot = dftot.append(pd.DataFrame(data["laureates"]),sort=False)

In [3]:
dftot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               949 non-null    object
 1   knownName        925 non-null    object
 2   givenName        925 non-null    object
 3   familyName       923 non-null    object
 4   fullName         925 non-null    object
 5   gender           925 non-null    object
 6   birth            924 non-null    object
 7   links            949 non-null    object
 8   nobelPrizes      949 non-null    object
 9   death            628 non-null    object
 10  orgName          24 non-null     object
 11  nativeName       24 non-null     object
 12  acronym          10 non-null     object
 13  founded          23 non-null     object
 14  penName          11 non-null     object
 15  birthCountry     1 non-null      object
 16  birthCountryNow  1 non-null      object
 17  birthContinent   1 non-null      ob

In [9]:
dftot.tail(20)


,id,knownName,givenName,familyName,fullName,gender,birth,links,nobelPrizes,death,orgName,nativeName,acronym,founded,penName,birthCountry,birthCountryNow,birthContinent
929,328,"{'en': 'William P. Murphy', 'se': 'William P. ...","{'en': 'William P.', 'se': 'William P.'}","{'en': 'Murphy', 'se': 'Murphy'}","{'en': 'William Parry Murphy', 'se': 'William ...",male,"{'date': '1892-02-06', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1934', 'category': {'en': 'Phy...","{'date': '1987-10-09', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
930,716,"{'en': 'William Vickrey', 'se': 'William Vickr...","{'en': 'William', 'se': 'William'}","{'en': 'Vickrey', 'se': 'Vickrey'}","{'en': 'William Vickrey', 'se': 'William Vickr...",male,"{'date': '1914-06-21', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1996', 'category': {'en': 'Eco...","{'date': '1996-10-11', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
931,63,"{'en': 'Willis E. Lamb', 'se': 'Willis E. Lamb'}","{'en': 'Willis E.', 'se': 'Willis E.'}","{'en': 'Lamb', 'se': 'Lamb'}","{'en': 'Willis Eugene Lamb', 'se': 'Willis Eug...",male,"{'date': '1913-07-12', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1955', 'category': {'en': 'Phy...","{'date': '2008-05-15', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
932,529,"{'en': 'Willy Brandt', 'se': 'Willy Brandt'}","{'en': 'Willy', 'se': 'Willy'}","{'en': 'Brandt', 'se': 'Brandt'}","{'en': 'Willy Brandt', 'se': 'Willy Brandt'}",male,"{'date': '1913-12-18', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1971', 'category': {'en': 'Pea...","{'date': '1992-10-08', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,624,"{'en': 'Winston Churchill', 'se': 'Winston Chu...","{'en': 'Winston', 'se': 'Winston'}","{'en': 'Churchill', 'se': 'Churchill'}",{'en': 'Sir Winston Leonard Spencer Churchill'...,male,"{'date': '1874-11-30', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1953', 'category': {'en': 'Lit...","{'date': '1965-01-24', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
934,673,"{'en': 'Wislawa Szymborska', 'se': 'Wislawa Sz...","{'en': 'Wislawa', 'se': 'Wislawa'}","{'en': 'Szymborska', 'se': 'Szymborska'}","{'en': 'Wislawa Szymborska', 'se': 'Wislawa Sz...",female,"{'date': '1923-07-02', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1996', 'category': {'en': 'Lit...","{'date': '2012-02-01', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
935,594,"{'en': 'Wladyslaw Reymont', 'se': 'Wladyslaw R...","{'en': 'Wladyslaw', 'se': 'Wladyslaw'}","{'en': 'Reymont', 'se': 'Reymont'}","{'en': 'Wladyslaw Stanislaw Reymont', 'se': 'W...",male,"{'date': '1867-05-07', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1924', 'category': {'en': 'Lit...","{'date': '1925-12-05', 'place': {'city': {'en'...",NaN,NaN,NaN,NaN,(pen-name of Rejment),NaN,NaN,NaN
936,663,"{'en': 'Wole Soyinka', 'se': 'Wole Soyinka'}","{'en': 'Wole', 'se': 'Wole'}","{'en': 'Soyinka', 'se': 'Soyinka'}","{'en': 'Wole Soyinka', 'se': 'Wole Soyinka'}",male,"{'date': '1934-07-13', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '1986', 'category': {'en': 'Lit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,739,"{'en': 'Wolfgang Ketterle', 'se': 'Wolfgang Ke...","{'en': 'Wolfgang', 'se': 'Wolfgang'}","{'en': 'Ketterle', 'se': 'Ketterle'}","{'en': 'Wolfgang Ketterle', 'se': 'Wolfgang Ke...",male,"{'date': '1957-10-21', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2001', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,137,"{'en': 'Wolfgang Paul', 'se': 'Wolfg

In [32]:
listNobel = []
for index,row in dftot.iterrows():
    new_item = dict()
    new_item['id'] = row['id']
    try:
        new_item['name_en'] = row['knownName']['en']
    except:
        pass
    try:
        new_item['name_se'] = row['knownName']['se']
    except:
        pass
    new_item['link'] = row['links']['href']
    try:
        new_item['gender'] = row['gender']
    except:
        pass
    listNobel.append(new_item)
NobelTot = pd.DataFrame(listNobel,
                  columns=['id','name_en','name_se','link','gender'])

NobelTot.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 949 entries, 0 to 948
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       949 non-null    object
 1   name_en  925 non-null    object
 2   name_se  925 non-null    object
 3   link     949 non-null    object
 4   gender   925 non-null    object
dtypes: object(5)
memory usage: 37.2+ KB


## EWS Vrandečić

In [37]:
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)] 

,id,name_en,name_se,link,gender


In [38]:
#NobelTot

In [34]:
print("End run: ", datetime.now())

End run:  2020-10-06 23:17:05.330020
